In [51]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [52]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000646717s, CPU 0.000654s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0181345s, CPU 0.010119s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [53]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:AxiSymmetric)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 16)


LowLevelFEM.Problem("body1-2D", :AxiSymmetric, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 98)

In [54]:
supp = FEM.displacementConstraint("left", uy=0)
load = FEM.load("right", fx=1)

("right", 1, 0, 0)

In [55]:
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [load])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :f2D)

In [56]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])

In [57]:
q1 = FEM.solveDisplacement(K, f)

LowLevelFEM.VectorField(Matrix{Float64}[], [9.482402621955039e-8; 0.0; … ; 2.5994609441980325e-6; -9.745989607018252e-7;;], [0.0], Int64[], 1, :u2D)

In [58]:
FEM.showDoFResults(problem, q1, :uvec, visible=true)

0

In [59]:
q2 = FEM.solveDisplacement(problem, [load], [supp])

LowLevelFEM.VectorField(Matrix{Float64}[], [9.482402621955039e-8; 0.0; … ; 2.5994609441980325e-6; -9.745989607018252e-7;;], [0.0], Int64[], 1, :u2D)

In [60]:
FEM.showDoFResults(problem, q2, :uvec)

1

In [61]:
elsupp = FEM.elasticSupport("left", kx=1e6, ky=1e6)

("left", 1.0e6, 1.0e6, 0)

In [62]:
q4 = FEM.solveDisplacement(problem, [load], [], [elsupp])

LowLevelFEM.VectorField(Matrix{Float64}[], [-5.518557609496582e-23; 9.054552941337081e-7; … ; 2.6213754931219642e-6; -1.5706097137120894e-6;;], [0.0], Int64[], 1, :u2D)

In [63]:
FEM.showDoFResults(problem, q4, :uvec)

2

In [64]:
q3 = copy(q2)
q3.a[1] = 5
q2.a[1] = 3
q3.a[1]

5.0

In [65]:
q2 + q3
q2 - q3
q2 * 2
2 * q2
q2 / 2

LowLevelFEM.VectorField(Matrix{Float64}[], [1.5; 0.0; … ; 1.2997304720990163e-6; -4.872994803509126e-7;;], [0.0], Int64[], 1, :u2D)

In [66]:
A1 = FEM.solveStrain(problem, q1)

LowLevelFEM.TensorField([[3.521321241915711e-6; -1.9344964189217016e-8; … ; 0.0; 3.4675622061253977e-6;;], [3.52552792090209e-6; -2.3943325671370416e-8; … ; 0.0; 3.465151654427786e-6;;], [3.537480773224319e-6; 1.1755991649166643e-8; … ; 0.0; 3.4529687493902648e-6;;], [3.4865567547400394e-6; -3.374462459164065e-7; … ; 0.0; 3.2079852751169067e-6;;], [3.531900331356171e-6; 1.0010898386934864e-8; … ; 0.0; 3.4616469928091556e-6;;], [3.3032845102210406e-6; -2.0934706120525767e-7; … ; 0.0; 3.0902974050036788e-6;;], [3.5338241685441784e-6; -3.5455880042146624e-9; … ; 0.0; 3.471164594388493e-6;;], [3.5859640773687216e-6; 7.087529149366037e-7; … ; 0.0; 3.0921966026839223e-6;;], [3.6007017717707185e-6; 1.5279501441868856e-7; … ; 0.0; 3.2639366066559108e-6;;], [3.298760829647533e-6; 7.080164282638098e-7; … ; 0.0; 2.8963973985811887e-6;;]  …  [3.603543945240217e-6; -3.6827999290408716e-7; … ; 0.0; 3.4615642064369165e-6;;], [3.5020787267942713e-6; -1.863099657303845e-8; … ; 0.0; 3.4761245835837836e-

In [67]:
A2 = FEM.solveStrain(problem, q1, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [3.3618534353672495e-6; 6.915791801805019e-6; … ; 0.0; 3.47074020054166e-6;;], [0.0], Int64[], 1, :e)

In [68]:
FEM.showStrainResults(problem, A1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


3

In [69]:
FEM.showDoFResults(problem, A2, :e)

4

In [70]:
A3 = FEM.elementsToNodes(problem, A1)

LowLevelFEM.TensorField(Matrix{Float64}[], [3.3618534353672495e-6; 3.4578959009025097e-6; … ; 0.0; 3.47074020054166e-6;;], [0.0], Int64[], 1, :e)

In [71]:
FEM.showDoFResults(problem, A3, :e)

5

In [72]:
S1 = FEM.solveStress(problem, q1)

LowLevelFEM.TensorField([[1.0007816986003326; -0.002976148336802618; … ; 0.0; 0.9927892182900088;;], [1.0026826387799341; -0.0036835885648262177; … ; 0.0; 0.9931158418187782;;], [1.0040008799206712; 0.0018086140998717912; … ; 0.0; 0.9899976943209448;;], [1.0182738526816022; -0.05191480706406254; … ; 0.0; 1.010360230418573;;], [1.002629274533723; 0.0015401382133745944; … ; 0.0; 0.9928224043825158;;], [0.9920061557218951; -0.03220724018542426; … ; 0.0; 0.9806030675851506;;], [1.0016686507612713; -0.0005454750775714865; … ; 0.0; 0.9922543801434888;;], [1.0447806767630978; 0.10903890999024672; … ; 0.0; 0.9914086657465211;;], [1.025220891547539; 0.023506925295182855; … ; 0.0; 0.9519103802624226;;], [1.0009319441830142; 0.10892560434827843; … ; 0.0; 0.9164376620071073;;]  …  [1.0491498767439236; -0.05665846044678264; … ; 0.0; 1.038228358374439;;], [0.99807737000123; -0.0028663071650828387; … ; 0.0; 0.9936236452591033;;], [0.9978082428735087; 0.0028085497698175903; … ; 0.0; 0.995025288315958;

In [73]:
S2 = FEM.solveStress(problem, q1, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.8216722333430653; 0.5319839847542323; … ; 0.0; 0.9941502236238554;;], [0.0], Int64[], 1, :s)

In [74]:
FEM.showStressResults(problem, S1, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


6

In [75]:
FEM.showDoFResults(problem, S2, :s)

7

In [76]:
S3 = FEM.elementsToNodes(problem, S1)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.8216722333430653; 0.5319839847542323; … ; 0.0; 0.9941502236238554;;], [0.0], Int64[], 1, :s)

In [77]:
FEM.showDoFResults(problem, S3, :s)

8

In [78]:
thermo = FEM.Problem([mat], type=:PlaneHeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 16)
Info    : Renumbering nodal model data (9 steps)


LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 98)

In [79]:
temp = FEM.temperatureConstraint("left", T=0)
flux = FEM.heatFlux("right", qn=1)

flx = FEM.heatFluxVector(thermo, [flux])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; -0.062499999999999924; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :qn)

In [80]:
KT = FEM.heatConductionMatrix(thermo)

FEM.applyBoundaryConditions!(thermo, KT, flx, [temp])

T1 = FEM.solveTemperature(KT, flx)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; -0.02222222222222229; … ; -0.005781351831909781; -0.016643653924583945;;], [0.0], Int64[], 1, :T)

In [81]:
FEM.showDoFResults(thermo, T1, :T)

9

In [82]:
T2 = FEM.solveTemperature(thermo, [flux], [temp])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; -0.02222222222222229; … ; -0.005781351831909781; -0.016643653924583945;;], [0.0], Int64[], 1, :T)

In [83]:
FEM.showDoFResults(thermo, T2, :T)

10

In [84]:
qT1 = FEM.solveHeatFlux(thermo, T2)

LowLevelFEM.VectorField([[0.9999999999999989; -1.2490009027033011e-15; … ; 0.9999999999999989; -1.2490009027033011e-15;;], [0.9999999999999973; 1.2490009027033011e-15; … ; 0.9999999999999973; 1.2490009027033011e-15;;], [1.0000000000000002; -8.118505867571457e-15; … ; 1.0000000000000002; -8.118505867571457e-15;;], [1.0000000000000024; -4.683753385137379e-16; … ; 1.0000000000000024; -4.683753385137379e-16;;], [1.000000000000004; -2.4980018054066022e-15; … ; 1.000000000000004; -2.4980018054066022e-15;;], [1.000000000000003; 7.806255641895632e-17; … ; 1.000000000000003; 7.806255641895632e-17;;], [0.9999999999999989; 7.494005416219807e-15; … ; 0.9999999999999989; 7.494005416219807e-15;;], [1.000000000000004; -3.122502256758253e-16; … ; 1.000000000000004; -3.122502256758253e-16;;], [1.0000000000000042; -7.806255641895632e-16; … ; 1.0000000000000042; -7.806255641895632e-16;;], [1.0000000000000044; -3.122502256758253e-16; … ; 1.0000000000000044; -3.122502256758253e-16;;]  …  [1.000000000000002

In [85]:
FEM.showElementResults(thermo, qT1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


11

In [86]:
qT2 = FEM.solveHeatFlux(thermo, T2, DoFResults=true)

LowLevelFEM.VectorField(Matrix{Float64}[], [1.000000000000004; 7.806255641895632e-17; … ; 1.0000000000000027; 1.0408340855860843e-15;;], [0.0], Int64[], 1, :q2D)

In [87]:
FEM.showDoFResults(thermo, qT2, :qvec)

12

In [88]:
qT3 = FEM.elementsToNodes(thermo, qT1)

LowLevelFEM.VectorField(Matrix{Float64}[], [1.000000000000004; 7.806255641895632e-17; … ; 1.0000000000000027; 1.0408340855860843e-15;;], [0.0], Int64[], 1, :q2D)

In [89]:
FEM.showDoFResults(thermo, qT3, :qvec)

13

In [90]:
heatconvB = FEM.heatConvection("bottom", h=1, Tₐ=0)
heatconvT = FEM.heatConvection("top", h=100, Tₐ=20)

("top", 100, 20, 0 + 2im)

In [91]:
T3 = FEM.solveTemperature(thermo, [], [temp], [heatconvB, heatconvT])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 6.028905064944964; … ; 6.149721965107743; 10.661603694875508;;], [0.0], Int64[], 1, :T)

In [92]:
FEM.showDoFResults(thermo, T3, :T)

14

In [93]:
fT = FEM.thermalLoadVector(problem, T1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.00012298077901520512; 0.00013635384781205732; … ; 0.005981029859801124; 2.0816681711721685e-17;;], [0.0], Int64[], 1, :f2D)

In [94]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])
q5 = FEM.solveDisplacement(K, f)

LowLevelFEM.VectorField(Matrix{Float64}[], [9.482402621955039e-8; 0.0; … ; 2.5994609441980325e-6; -9.745989607018252e-7;;], [0.0], Int64[], 1, :u2D)

In [95]:
ST1 = FEM.solveStress(problem, q5, T=T1)

LowLevelFEM.TensorField([[1.1027373319604712; -0.002976148336802618; … ; 0.0; 1.1116887948936018;;], [1.1215822153835273; -0.0036835885648262177; … ; 0.0; 1.0950714751789168;;], [1.1211663277936685; 0.0018086140998717912; … ; 0.0; 1.0907728736752176;;], [1.0340422106951745; -0.05191480706406254; … ; 0.0; 1.0436935637519063;;], [1.103404453887996; 0.0015401382133745944; … ; 0.0; 1.109987852255513;;], [1.0077745137354674; -0.03220724018542426; … ; 0.0; 1.0056125413555745;;], [1.1350019840946053; -0.0005454750775714865; … ; 0.0; 1.1255877134768224;;], [1.0593787023882588; 0.10903890999024672; … ; 0.0; 1.0151250949961372;;], [1.0665398053560253; 0.023506925295182855; … ; 0.0; 0.9835861394848056;;], [1.0246483734326304; 0.10892560434827843; … ; 0.0; 0.9310356876322682;;]  …  [1.0741498767439237; -0.05665846044678264; … ; 0.0; 1.079895025041106;;], [1.1063400805132158; -0.0028663071650828387; … ; 0.0; 1.1130770433629182;;], [1.1150389713050173; 0.0028085497698175903; … ; 0.0; 1.1027781254917

In [96]:
ST2 = FEM.solveStress(problem, q5, T=T1, T₀=T1)

LowLevelFEM.TensorField([[1.0007816986003326; -0.002976148336802618; … ; 0.0; 0.9927892182900088;;], [1.0026826387799341; -0.0036835885648262177; … ; 0.0; 0.9931158418187782;;], [1.0040008799206712; 0.0018086140998717912; … ; 0.0; 0.9899976943209448;;], [1.0182738526816022; -0.05191480706406254; … ; 0.0; 1.010360230418573;;], [1.002629274533723; 0.0015401382133745944; … ; 0.0; 0.9928224043825158;;], [0.9920061557218951; -0.03220724018542426; … ; 0.0; 0.9806030675851506;;], [1.0016686507612713; -0.0005454750775714865; … ; 0.0; 0.9922543801434888;;], [1.0447806767630978; 0.10903890999024672; … ; 0.0; 0.9914086657465211;;], [1.025220891547539; 0.023506925295182855; … ; 0.0; 0.9519103802624226;;], [1.0009319441830142; 0.10892560434827843; … ; 0.0; 0.9164376620071073;;]  …  [1.0491498767439236; -0.05665846044678264; … ; 0.0; 1.038228358374439;;], [0.99807737000123; -0.0028663071650828387; … ; 0.0; 0.9936236452591033;;], [0.9978082428735087; 0.0028085497698175903; … ; 0.0; 0.995025288315958;

In [97]:
FEM.showElementResults(problem, ST1, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


15

In [98]:
FEM.showElementResults(problem, ST2, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


16

In [99]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [100]:
gmsh.finalize()